In [2]:
!pip install catboost


from google.colab import drive
drive.mount('./gdrive')
from pathlib import Path

data_dir = Path('/content/gdrive/MyDrive/MAIL')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
from sklearn.metrics import *
from catboost import CatBoostClassifier
import numpy as np
from sklearn.model_selection import StratifiedKFold


warnings.filterwarnings('ignore')

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [4]:
raw_data = pd.read_csv(data_dir / 'train_id.csv', index_col=0)
vals = raw_data.Data.apply(eval)
res = [val if len(val) == 240 else val + [0] * (240 - len(val)) for val in vals]
vals1 = raw_data.Data_2.apply(eval)
res1 = [val if len(val) == 240 else val + [0] * (240 - len(val)) for val in vals1]

df_1 = pd.DataFrame(res).add_prefix('data')
df_2 = pd.DataFrame(res1).add_prefix('data_new')

additional_data = pd.concat((df_1, df_2), axis=1)


In [5]:
tt = pd.concat((additional_data, raw_data), axis=1)

In [6]:
train_feat, test_feat = train_test_split(tt, random_state=142, test_size=0.2, stratify=raw_data.Class_label)
train_feat.reset_index(drop=True)

,data0,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data_new238,data_new239,Filename,Test_index,Presentation,Question,Data,Data_2,Class_label,id
0,-41,-62,-49,-26,-15,-6,7,22,43,63,...,-728,-817,80508871-29ee-4d07-8b0a-e563731ce6d1,1,3,2,"[-41, -62, -49, -26, -15, -6, 7, 22, 43, 63, 3...","[-704, -743, -729, -702, -689, -678, -663, -64...",2,262
1,-1000,-1178,-1134,-984,-885,-860,-837,-774,-587,-594,...,884,479,f0e0b261-352b-4207-b906-afdae32afa6a,2,3,11,"[-1000, -1178, -1134, -984, -885, -860, -837, ...","[-1589, -1974, -1994, -1793, -1638, -1633, -16...",1,165
2,-734,-692,-654,-621,-559,-480,-404,-339,-280,-234,...,-766,-709,4ab63cfc-7291-4c7c-a4c8-437212605677,0,1,9,"[-734, -692, -654, -621, -559, -480, -404, -33...","[-1491, -1419, -1377, -1358, -1287, -1187, -11...",1,319
3,-78,-61,-42,-45,-59,-66,-50,-16,19,34,...,1150,1130,12555fb4-ea91-428f-955e-bc84d4e5cb74,2,2,8,"[-78, -61, -42, -45, -59, -66, -50, -16, 19, 3...","[868, 825, 791, 794, 813, 812, 775, 697, 623, ...",1,293
4,-202,-148,-132,-89,-20,61,138,196,80,-325,...,118,162,73d5adee-c1ec-46b3-a519-ccf381293116,0,3,4,"[-202, -148, -132, -89, -20, 61, 138, 196, 80,...","[-123, -95, -81, -33, 35, 101, 154, 186, 28, -...",0,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25101,-41,-73,-86,-86,-81,-78,-77,-73,-60,-44,...,-335,-293,07f86761-78ba-4d41-af89-3bdbbfec2b00,0,1,8,"[-41, -73, -86, -86, -81, -78, -77, -73, -60, ...","[-495, -537, -558, -558, -548, -542, -538, -53...",1,236
25102,54,65,76,86,89,-60,-148,-144,-103,-77,...,558,517,fcbe8631-e21c-41d6-975f-6de80d663bcf,2,3,9,"[54, 65, 76, 86, 89, -60, -148, -144, -103, -7...","[1088, 1032, 979, 914, 1011, 1451, 1584, 1565,...",0,283
25103,803,807,830,853,865,878,893,908,919,927,...,-416,-478,0cc70727-e962-49ef-a3f2-c3956bc7100b,3,2,2,"[803, 807, 830, 853, 865, 878, 893, 908, 919, ...","[-102, -96, -206, -348, -424, -502, -582, -652...",2,227
25104,-447,-438,-421,-404,-390,-374,-361,-356,-395,-471,...,72,20,265036e7-5434-4417-a116-e28ea903e1c7,0,3,5,"[-447, -438, -421, -404, -390, -374, -361, -35...","[-18, -19, -21, -22, -22, -22, -24, -24, -16, ...",1,150


In [7]:
def drop_smth(df):
  df = df.drop('id', axis=1)
  df = df.drop('Data', axis=1)
  df = df.drop('Data_2', axis=1)
  df = df.drop('Filename', axis=1)
  return df
train_feat = drop_smth(train_feat)
test_feat = drop_smth(test_feat)


In [8]:
train_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25106 entries, 16833 to 4677
Columns: 484 entries, data0 to Class_label
dtypes: int64(484)
memory usage: 92.9 MB


In [346]:
print("Train_feat: ",train_feat.shape)


Train_feat:  (25106, 484)
gen_feat_data:  (23885, 240)
gen_feat_data_2:  (23885, 240)


In [19]:

params = {
    "loss_function": "MultiClass",
    "verbose": 100,
    'eval_metric': 'TotalF1',
    'use_best_model': True, 
    'early_stopping_rounds': 300,
    'iterations': 4000,
    # 'eval_metric': 'TotalF1',
}


def oof_fit(data):
    kf = StratifiedKFold(n_splits=2, random_state=142, shuffle=True)
    models = []
    ginnis = []
    for train_ids, val_ids in kf.split(data.drop('Class_label', axis=1), data.Class_label):
        train_x, train_y = data.iloc[train_ids].drop('Class_label', axis=1), data.iloc[train_ids].Class_label
        val_x, val_y = data.iloc[val_ids].drop('Class_label', axis=1), data.iloc[val_ids].Class_label
        
        model = CatBoostClassifier(**params)
        model.fit(train_x, train_y, eval_set=(val_x, val_y))
        models.append(model)
        roc = roc_auc_score(val_y, model.predict_proba(val_x),multi_class='ovr')
        ginnis.append(roc * 2 - 1)
        print(f"MultiClass = {ginnis[-1]}")
    print(f"Mean ginny = {np.mean(ginnis)}")
    return models

In [10]:
models = oof_fit(train_feat)

Learning rate set to 0.114236
0:	learn: 0.4054399	test: 0.4043941	best: 0.4043941 (0)	total: 837ms	remaining: 13m 56s
100:	learn: 0.4139821	test: 0.4052252	best: 0.4057922 (98)	total: 53.1s	remaining: 7m 53s
200:	learn: 0.4748927	test: 0.4102950	best: 0.4108956 (193)	total: 1m 50s	remaining: 7m 19s
300:	learn: 0.5428979	test: 0.4153814	best: 0.4156389 (286)	total: 2m 41s	remaining: 6m 14s
400:	learn: 0.6129964	test: 0.4259310	best: 0.4275478 (378)	total: 3m 40s	remaining: 5m 28s
500:	learn: 0.6772940	test: 0.4336791	best: 0.4337029 (498)	total: 4m 21s	remaining: 4m 20s
600:	learn: 0.7286912	test: 0.4401198	best: 0.4401198 (600)	total: 5m 2s	remaining: 3m 21s
700:	learn: 0.7711744	test: 0.4456709	best: 0.4469882 (695)	total: 6m 4s	remaining: 2m 35s
800:	learn: 0.8070514	test: 0.4502387	best: 0.4505514 (799)	total: 6m 56s	remaining: 1m 43s
900:	learn: 0.8376088	test: 0.4495266	best: 0.4511421 (892)	total: 7m 41s	remaining: 50.7s
999:	learn: 0.8637859	test: 0.4541509	best: 0.4541509 (999)

In [12]:
y_pred = models[-1].predict(test_feat.drop('Class_label', axis=1))
y_val = test_feat.Class_label
df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred.ravel()})
df

,Actual,Predicted
20412,1,1
30783,0,1
11979,2,1
26905,0,0
16614,0,1
...,...,...
2432,1,1
28846,1,1
5541,1,1
7825,0,1


In [13]:
import collections

print(collections.Counter(y_val))
print(collections.Counter(y_pred.ravel()))


Counter({1: 3528, 2: 1406, 0: 1343})
Counter({1: 5737, 2: 290, 0: 250})


In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred)

confmtrx = np.array(confusion_matrix(y_val, y_pred))
pd.DataFrame(confmtrx, index=['0','1', '2'],
columns=['0', '1', '2'])

,0,1,2
0,112,1196,35
1,99,3282,147
2,39,1259,108


In [15]:
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

print('Accuracy Score:', metrics.accuracy_score(y_val, y_pred))  
#Create classification report
class_report=classification_report(y_val, y_pred)
print(class_report)

Accuracy Score: 0.5579098295364028
              precision    recall  f1-score   support

           0       0.45      0.08      0.14      1343
           1       0.57      0.93      0.71      3528
           2       0.37      0.08      0.13      1406

    accuracy                           0.56      6277
   macro avg       0.46      0.36      0.33      6277
weighted avg       0.50      0.56      0.46      6277



In [16]:
from sklearn.metrics import f1_score
print(f1_score(y_val, y_pred, average='macro'))

0.32548214297520356


In [ ]:
models1 = oof_fit(train_feat)

Learning rate set to 0.066883
0:	learn: 0.4071483	test: 0.4055292	best: 0.4055292 (0)	total: 544ms	remaining: 36m 14s
100:	learn: 0.4056967	test: 0.4045812	best: 0.4055292 (0)	total: 37.8s	remaining: 24m 19s
200:	learn: 0.4282198	test: 0.4055791	best: 0.4059743 (195)	total: 1m 15s	remaining: 23m 38s
300:	learn: 0.4793608	test: 0.4096133	best: 0.4103599 (290)	total: 1m 54s	remaining: 23m 23s
400:	learn: 0.5321748	test: 0.4151804	best: 0.4151804 (400)	total: 2m 32s	remaining: 22m 45s
500:	learn: 0.5909714	test: 0.4200017	best: 0.4200068 (498)	total: 3m 10s	remaining: 22m 9s
600:	learn: 0.6399978	test: 0.4252988	best: 0.4254618 (570)	total: 3m 50s	remaining: 21m 42s
700:	learn: 0.6881591	test: 0.4325006	best: 0.4327061 (697)	total: 4m 28s	remaining: 21m 3s
800:	learn: 0.7286257	test: 0.4372725	best: 0.4375358 (772)	total: 5m 6s	remaining: 20m 24s
900:	learn: 0.7657582	test: 0.4376910	best: 0.4379446 (893)	total: 5m 46s	remaining: 19m 52s
1000:	learn: 0.7990069	test: 0.4394100	best: 0.4395

In [ ]:
def generate_features(df, 
                      feature_name="Data",
                      new_feature_name="gen_feature_",
                      feature_count=240):
  df[feature_name] = df[feature_name].str.replace("[", "")
  df[feature_name] = df[feature_name].str.replace("]", "")
    
  # new data frame with split value columns
  #new = df[feature_name].str.split(", ", n = feature_count, expand = True).astype(str).astype(int)
  #trans_names = [new_feature_name + feature_name + "_" + str(i) for i in range(0,feature_count)]
  #res = pd.DataFrame(new.values, columns=trans_names)
  #for column in res:
    #res[column] = res[column].to_numpy(dtype='int')

  res_df.isna().sum().max()
  return res.astype('int32')

def prepare_df(df):
  df['Test_index'] = df['Test_index'].to_numpy(dtype='int')
  df['Presentation'] = df['Presentation'].to_numpy(dtype='int')
  df['Question'] = df['Question'].to_numpy(dtype='int')
  df.drop(df[df.Data_2.str.count(",")==0].index, inplace=True)
  df.drop(df[df.Data_2.str.count(",")==138].index, inplace=True)
  df.drop(df[df.Data.str.count(",")==138].index, inplace=True)
  return df